In [1]:
import sys
sys.path.append('..')

In [2]:
import json
from pathlib import Path
from collections import Counter

from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

from utils.candidate_labels import CANDIDATE_LABELS

/home/workspace/notion-auto-archive/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

dataset = load_dataset("xnli", "en")

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 872.54it/s]


Dataset xnli downloaded and prepared to /root/.cache/huggingface/datasets/xnli/en/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 366.59it/s]


In [15]:
type(dataset["train"][0]["label"])

int

In [28]:
ROOT_DIR = Path.cwd().parent

In [29]:
df_qiita = pd.read_json(ROOT_DIR / "data/qiita.json")
df_zenn = pd.read_json(ROOT_DIR / "data/zenn.json")

In [30]:
df_raw = pd.concat([df_qiita, df_zenn], axis=0)

In [31]:
hypothesis_template = "This text is about {}."

In [34]:
df = pd.DataFrame(
    columns=["premise", "hypothesis", "label"],
)
idx = 0
for row in df_raw.itertuples():
    if not row.cleansed_content:
        continue
    
    categories = []
    for tag in row.tags:
        if tag in CANDIDATE_LABELS.keys():
            categories.append(CANDIDATE_LABELS[tag])
    
    for category in categories:
        df.at[idx, "premise"] = row.cleansed_content
        df.at[idx, "hypothesis"] = hypothesis_template.format(category)
        df.at[idx, "label"] = 0
        idx += 1

In [36]:
le = []
for row in df.iterrows():
    le.append(len(row[1]["premise"]))